<a href="https://colab.research.google.com/github/Vsanku01/Deep-Learning-Tuts/blob/master/Plant_Pathology_2020_FGVC7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [9]:
import tensorflow
print(tensorflow.__version__)

1.15.0


In [1]:
!pip install efficientnet

In [0]:
import pandas as pd
import numpy as np

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/plant-pathology-2020-fgvc7/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/plant-pathology-2020-fgvc7/test.csv')

In [0]:
train_df['image_id']=train_df['image_id']+'.jpg'

In [75]:
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0.jpg,0,0,0,1
1,Train_1.jpg,0,1,0,0
2,Train_2.jpg,1,0,0,0
3,Train_3.jpg,0,0,1,0
4,Train_4.jpg,1,0,0,0


In [0]:
test_df['image_id']=test_df['image_id']+'.jpg'

In [12]:
test_df.head()

,image_id
0,Test_0.jpg
1,Test_1.jpg
2,Test_2.jpg
3,Test_3.jpg
4,Test_4.jpg


In [16]:
train_df.shape


(1821, 5)

In [17]:
test_df.shape

(1821, 1)

# Train ,Test ,Validation Split

In [0]:
from sklearn.model_selection import train_test_split
train,val = train_test_split(train_df,test_size = 0.2)

In [77]:
len(train_df),len(train),len(val)

(1821, 1456, 365)

In [78]:
val.head()

,image_id,healthy,multiple_diseases,rust,scab
343,Train_343.jpg,0,0,0,1
1490,Train_1490.jpg,0,0,1,0
696,Train_696.jpg,0,0,0,1
385,Train_385.jpg,0,0,1,0
1551,Train_1551.jpg,0,0,1,0


# Image Data Generator

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_data_gen = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    zoom_range = 0.1,
    rotation_range = 2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    rescale = 1 /255,
    fill_mode = 'nearest',
    shear_range = 0.2,
    brightness_range = [0.5,1.5]
)

In [0]:
image_shape = 300
batch_size = 16

In [82]:
train_generator = train_data_gen.flow_from_dataframe(
    train,
    directory='/content/drive/My Drive/plant-pathology-2020-fgvc7/images',
    target_size = (image_shape,image_shape),
    x_col = "image_id",
     y_col=['healthy','multiple_diseases','rust','scab'],
    shuffle = False,
    subset = 'training',
    class_mode='raw',
    batch_size = batch_size
    )

Found 1456 validated image filenames.


In [83]:
val_generator = train_data_gen.flow_from_dataframe(
    val,
    directory='/content/drive/My Drive/plant-pathology-2020-fgvc7/images',
    target_size = (image_shape,image_shape),
    x_col = 'image_id',
    y_col = ['healthy','multiple_diseases','rust','scab'],
    class_mode = 'raw',
    shuffle = False,
    subset = 'training',
    batch_size = batch_size
    
    )

Found 365 validated image filenames.


In [84]:
test_generator=train_data_gen.flow_from_dataframe(
    test_df,
    directory='/content/drive/My Drive/plant-pathology-2020-fgvc7/images',
    target_size=(image_shape,image_shape),
                                                      x_col="image_id",
                                                      y_col=None,
                                                      class_mode=None,
                                                      shuffle=False,
                                                      batch_size=batch_size)

Found 1821 validated image filenames.


# Model

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,GlobalAveragePooling2D,GlobalMaxPooling2D,concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
import efficientnet.keras as efn 
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score
from tensorflow.keras import callbacks
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [0]:
pretrained_model = tf.keras.applications.VGG16(weights='imagenet',include_top=False,input_shape=(image_shape,image_shape,3))
pretrained_model.trainable = False  # False = transfer learning, True = fine-tuning



model = Sequential([
    pretrained_model,
    GlobalAveragePooling2D(),
    Dense(64,activation='relu'),
    Dense(4,activation='softmax')
])

In [0]:
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

In [54]:
y_true=val.iloc[:,1::].values
y_true

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       ...,
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 0, 1]])

In [0]:
results = model.fit_generator(
    train_generator,
    epochs=20,
    validation_data=val_generator,
  )

Epoch 1/20
21/91 [=====>........................] - ETA: 8:21 - loss: 1.2735 - acc: 0.3512